In [ ]:
import sys
import os
if os.path.exists('/kaggle/input/'):
    sys.path.append('/kaggle/input/bird-clef-utils')

import torch
torch.manual_seed(42)  # For reproducibility

from utils import ROOT, DURATIONS_FILE, LABEL_MAP_FILE, VOICE_SEGMENTS_FILE, MODEL_OUTPUT_PATH

# Dataset processing

In [ ]:
from utils import contains_human_voice, load_all_training_chunks, segment_length

training_chunks = load_all_training_chunks()
training_chunks['contains_human_voice'] = training_chunks.apply(
    lambda row: contains_human_voice(row['filename'], row['audio_start'], row['audio_start'] + segment_length),
    axis=1
)
training_chunks = training_chunks[~training_chunks['contains_human_voice']]


# define dataset & model & split data

In [ ]:
from utils import AudioDataset, EffNetB0Classifier

my_ds = AudioDataset(training_chunks[:10000])
model = EffNetB0Classifier(num_classes=206)

In [ ]:
from torch.utils.data import random_split

# Calculate lengths for train and validation splits
total_length = len(my_ds)
train_length = int(0.10 * total_length)
val_length = int(0.01 * total_length)
test_length = total_length - train_length - val_length

# Perform the split
train_ds, val_ds, _ = random_split(my_ds, [train_length, val_length, test_length])

print(f"Train dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")

# Train/Val

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch
from torch import nn

import torch.optim as optim

# Define DataLoader
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())  # Ensure labels are float for BCEWithLogitsLoss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels.float())  # Ensure labels are float for BCEWithLogitsLoss

            val_loss += loss.item()

            # Calculate accuracy
            predicted = (torch.sigmoid(outputs) > 0.5).int()
            total += labels.size(0) * labels.size(1)  # Total number of elements
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


In [ ]:
torch.save(model.state_dict(), MODEL_OUTPUT_PATH)